# Importing Packages

In [78]:
import requests
import json
from io import StringIO
from bs4 import BeautifulSoup as bs
from datetime import date, timedelta
import covidcast

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

import pickle

from sklearn.pipeline import Pipeline 
from sklearn.impute import KNNImputer, SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier



# Data Collection

- Dataset was orignally constructed (webscraping/api calls) made and relevant to classification.  At least 1000 rows and at least 10 features.

# Exploratory Data Analysis

- Student created at least 2 well-constructed visualizations, discussed descriptive statistics, and is able to interpret and compare or link them together to give a cohesive introduction to the data


## Geopandas File for Mapping

In [33]:
data = gpd.read_file('https://services9.arcgis.com/6Hv9AANartyT7fJW/arcgis/rest/services/USCounties_cases_V1/FeatureServer/0/query?where=1%3D1&outFields=Countyname,FIPS,Shape__Area,Shape__Length,ST_Name,ST_Abbr,ST_ID,OBJECTID&outSR=4326&f=json')

## NYT Mask Wearing Survey Data

- This data comes from a large number of interviews conducted online by the global data and survey firm Dynata at the request of The New York Times. 
- The firm asked a question about mask use to obtain 250,000 survey responses between July 2 and July 14, enough data to provide estimates more detailed than the state level. - (Several states have imposed new mask requirements since the completion of these interviews.)

Specifically, each participant was asked: 

*How often do you wear a mask in public when you expect to be within six feet of another person?*

This survey was conducted a single time, and at this point we have no plans to update the data or conduct the survey again.

The fields have the following definitions:

- **COUNTYFP**: The county FIPS code.
- **NEVER**: The estimated share of people in this county who would say never in response to the question: “How often do you wear a mask in public when you expect to be within six feet of another person?”
- **RARELY**: The estimated share of people in this county who would say rarely
- **SOMETIMES**: The estimated share of people in this county who would say sometimes
- **FREQUENTLY**: The estimated share of people in this county who would say frequently
- **ALWAYS**: The estimated share of people in this county who would say always


In [35]:
url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/mask-use/mask-use-by-county.csv'
s = requests.get(url).text
nymask = pd.read_csv(StringIO(s))
nymask.COUNTYFP = nymask.COUNTYFP.astype(str)
nymask.COUNTYFP = np.where(nymask['COUNTYFP'].str.len() == 4, '0' + nymask.COUNTYFP, nymask.COUNTYFP)
nymask.columns = ['FIPS', 'NEVER', 'RARELY', 'SOMETIMES', 'FREQUENTLY', 'ALWAYS']
nymask.head()

,FIPS,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,01001,0.053,0.074,0.134,0.295,0.444
1,01003,0.083,0.059,0.098,0.323,0.436
2,01005,0.067,0.121,0.120,0.201,0.491
3,01007,0.020,0.034,0.096,0.278,0.572
4,01009,0.053,0.114,0.180,0.194,0.459


In [ ]:
# fig, ax = plt.subplots(1, figsize=(20,20))
# merged.plot(column='trump_pct', cmap='RdBu_r', linewidth=0.8, ax=ax, edgecolor='black')
# # ax.axis('off')
# # ax.set_title('Percentage By County for 2016 Election', fontsize=18)
# # plt.savefig('images/election2016.png')

<img src='images/fb_mask_data.png'>

## Carnegie Mellon Mask Wearing Facebook Survey Data

Between the new survey’s deployment on September 8, 2020, and October 7th, the study collected 1,220,000 valid responses from respondents across the United States. 
Each response includes questions about symptoms, mask wearing, testing, and the other important topics described above, along with demographic details about the respondent. These demographics include age, gender, race, occupation, and education, allowing us to understand how different groups have been affected and which groups are currently most vulnerable to COVID-19. 

- **smoothed_wearing_mask**
  - Estimated percentage of people who wore a mask for most or all of the time while in public in the past 5 days; those not in public in the past 5 days are not counted.	

- **smoothed_others_masked**
  - Estimated percentage of respondents who say that most or all other people wear masks, when they are in public and social distancing is not possible


In [6]:
# create variable to hold date object for two days ago
two_days_ago = date.today() - timedelta(days=2)

In [7]:
# get the most recent survey data
mask_ind = covidcast.signal("fb-survey", "smoothed_wearing_mask",
                            date(2020, 10, 1), two_days_ago,
                            "county")
mask_oth = covidcast.signal('fb-survey', 'smoothed_others_masked', 
                            date(2020, 11, 24),two_days_ago, 
                            "county")

In [8]:
# remove data that isn't at the county level
mask_fip = mask_ind.loc[~mask_ind.geo_value.str.endswith('000')]
mask_oth = mask_oth.loc[~mask_oth.geo_value.str.endswith('000')]

In [38]:
mask_fip.head()

,geo_value,signal,time_value,issue,lag,value,stderr,sample_size,geo_type,data_source
1,01003,smoothed_wearing_mask,2020-10-01,2020-12-09,69,73.866307,2.968477,219.0684,county,fb-survey
2,01073,smoothed_wearing_mask,2020-10-01,2020-12-09,69,88.893937,1.822515,297.2281,county,fb-survey
3,01089,smoothed_wearing_mask,2020-10-01,2020-12-09,69,90.361381,1.421118,431.2580,county,fb-survey
4,01097,smoothed_wearing_mask,2020-10-01,2020-12-09,69,80.325870,2.119979,351.6316,county,fb-survey
5,01101,smoothed_wearing_mask,2020-10-01,2020-12-09,69,92.842875,2.164874,141.7822,county,fb-survey


In [13]:
mask_fip.describe()

,lag,value,stderr,sample_size
count,50264.000000,50264.000000,50264.000000,50264.000000
mean,25.046654,90.907979,1.861972,329.766106
std,22.055374,5.500603,0.864895,320.344010
min,1.000000,51.314471,0.252205,100.000000
25%,5.000000,88.127758,1.167788,143.917100
50%,16.000000,92.236727,1.739585,219.888050
75%,45.000000,94.972180,2.459793,399.257450
max,69.000000,99.677919,4.965824,4677.966900


In [48]:
df = pd.DataFrame(mask_fip.geo_value.unique())

In [54]:
df2 = mask_fip.groupby('geo_value').sample_size.sum()

In [55]:
df3 = mask_fip.groupby('geo_value').value.mean()

In [12]:
mask_oth.head(10)

,geo_value,signal,time_value,issue,lag,value,stderr,sample_size,geo_type,data_source
2,04013,smoothed_others_masked,2020-11-24,2020-12-09,15,82.923867,2.051720,336.3820,county,fb-survey
3,04019,smoothed_others_masked,2020-11-24,2020-12-09,15,84.678313,2.964860,147.5946,county,fb-survey
6,06037,smoothed_others_masked,2020-11-24,2020-12-09,15,87.485556,1.684146,386.0008,county,fb-survey
7,06059,smoothed_others_masked,2020-11-24,2020-12-09,15,81.656743,3.294606,137.9944,county,fb-survey
8,06065,smoothed_others_masked,2020-11-24,2020-12-09,15,87.018507,2.681319,157.1228,county,fb-survey
9,06071,smoothed_others_masked,2020-11-24,2020-12-09,15,89.023513,3.062219,104.2067,county,fb-survey
10,06073,smoothed_others_masked,2020-11-24,2020-12-09,15,86.891234,2.643510,162.9956,county,fb-survey
11,06085,smoothed_others_masked,2020-11-24,2020-12-09,15,93.406706,2.336948,112.7671,county,fb-survey
13,08041,smoothed_others_masked,2020-11-24,2020-12-09,15,83.909140,3.441874,113.9721,county,fb-survey
15,09001,smoothed_others_masked,2020-11-24,2020-12-09,15,94.998454,2.088127,108.9700,county,fb-survey


In [14]:
mask_oth.describe()

,lag,value,stderr,sample_size
count,22410.000000,22410.000000,22410.000000,22410.000000
mean,5.438108,82.779259,2.363568,334.298024
std,2.108070,11.656619,1.046377,330.008157
min,1.000000,24.554959,0.418443,100.000000
25%,5.000000,77.526024,1.509990,144.444525
50%,5.000000,86.256870,2.209437,218.304050
75%,5.000000,91.439472,3.178061,400.507175
max,15.000000,98.750000,4.992825,4374.027800


In [9]:
# check number of counties represented 
print(mask_ind.geo_value.value_counts().shape)
print(mask_fip.geo_value.value_counts().shape)
mask_oth.geo_value.value_counts().shape

(726,)
(677,)


(655,)

In [10]:
# pare down datasets to just columns of interest
mask_ind_means = pd.DataFrame(mask_fip.groupby(['geo_value'])['value'].mean()).reset_index()
mask_ind_means.columns = ['FIPS', 'ind_mask']
mask_ind_means['FIPS'] = mask_ind_means['FIPS'].astype(int)
# repeat for other survey
mask_oth_means = pd.DataFrame(mask_oth.groupby(['geo_value'])['value'].mean()).reset_index()
mask_oth_means.columns = ['FIPS', 'oth_mask']
mask_oth_means['FIPS'] = mask_oth_means['FIPS'].astype(int)

<img src='images/delphi_dec.png'>

In [44]:
mask_ind_means.head()

,FIPS,ind_mask
0,1003,79.671626
1,1051,88.291970
2,1069,85.591414
3,1073,91.454122
4,1081,89.816314


## Johns Hopkins COVID Data Geopandas

This feature layer contains the most up-to-date COVID-19 cases for the US. Data is pulled from the Coronavirus COVID-19 Global Cases by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University, the Red Cross, the Census American Community Survey, and the Bureau of Labor and Statistics, and aggregated at the US county level. 

This web map created and maintained by the Centers for Civic Impact at the Johns Hopkins University, and is supported by the Esri Living Atlas team and JHU Data Services. It is used in the COVID-19 United States Cases by County dashboard. 

 For more information on Johns Hopkins University’s response to COVID-19, visit the Johns Hopkins Coronavirus Resource Center where our experts help to advance understanding of the virus, inform the public, and brief policymakers in order to guide a response, improve care, and save lives.


In [74]:
data2 = gpd.read_file('https://opendata.arcgis.com/datasets/4cb598ae041348fb92270f102a6783cb_0.geojson')
print(data2.shape)
data2.head()

(3331, 88)


,OBJECTID,Countyname,ST_Name,ST_Abbr,ST_ID,FIPS,FatalityRa,Confirmedb,DeathsbyPo,PCTPOVALL_,Unemployme,Med_HH_Inc,State_Fata,DateChecke,EM_type,EM_date,EM_notes,url,Thumbnail,Confirmed,Deaths,Age_85,Age_80_84,Age_75_79,Age_70_74,Age_65_69,Beds_Licen,Beds_Staff,Beds_ICU,Ventilator,POP_ESTIMA,POVALL_201,Unemployed,Median_Hou,Recovered,Active,State_Conf,State_Deat,State_Reco,State_Test,AgedPop,NewCases,NewDeaths,TotalPop,NonHispWhP,BlackPop,AmIndop,AsianPop,PacIslPop,OtherPop,TwoMorPop,HispPop,Wh_Alone,Bk_Alone,AI_Alone,As_Alone,NH_Alone,SO_Alone,Two_More,Not_Hisp,Age_Less15,Age_15_24,Age_25_34,Age_Over75,Agetotal,NonHisp,Age_35_64,Age_65_74,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,NewCasebyP,Inpat_Occ,ICU_Occ,Shape__Area,Shape__Length,geometry
0,1,Autauga,Alabama,AL,01,01001,1.076426,8354.17,89.926440,13.8,3.6,118.9591,1.299898,01/07/2021 11:30:49,Govt Ordered Community Quarantine,4/3/2020 10:40:51 PM,AL Governor issued a Shelter In Place order.,https://bao.arcgis.com/covid-19/jhu/county/010...,https://coronavirus.jhu.edu/static/media/dashb...,4645,50,815,1026,1498,2440,2271,85,55,6,2,55601,7587,942,59338,0,0,384184,4994,0,1997125,8050,99,0,55200,41412,10475,159,568,5,41,1012,1528,42437,10565,159,568,32,409,1030,53672,10842,7192,7064,3339,55200,1528,22052,4711,99.0,210.0,31.0,37.0,29.0,49.0,26.0,59.0,40.0,36.0,30.0,9.0,48.0,53.0,178.054352,96.320346,100.000000,2.209382e+09,246839.865479,"POLYGON ((-86.41312 32.70739, -86.41219 32.526..."
1,2,Baldwin,Alabama,AL,01,01003,1.166758,6722.26,78.432452,9.8,3.6,115.4508,1.299898,01/07/2021 11:30:49,Govt Ordered Community Quarantine,4/3/2020 10:41:19 PM,AL Governor issued a Shelter In Place order.,https://bao.arcgis.com/covid-19/jhu/county/010...,https://coronavirus.jhu.edu/static/media/dashb...,14656,171,3949,4792,7373,11410,13141,386,362,51,8,218022,21069,3393,57588,0,0,384184,4994,0,1997125,40665,216,2,208107,172768,19529,1398,1668,9,410,2972,9353,179526,19764,1522,1680,9,2034,3572,198754,37621,23497,23326,16114,208107,9353,82998,24551,216.0,253.0,123.0,109.0,132.0,222.0,209.0,220.0,210.0,137.0,117.0,42.0,145.0,200.0,99.072571,74.029401,120.089286,5.770469e+09,728445.072448,"MULTIPOLYGON (((-87.78878 31.29877, -87.78849 ..."
2,3,Barbour,Alabama,AL,01,01005,2.191609,6418.55,140.669587,30.9,5.2,68.9280,1.299898,01/07/2021 11:30:49,Govt Ordered Community Quarantine,4/3/2020 10:43:21 PM,AL Governor issued a Shelter In Place order.,https://bao.arcgis.com/covid-19/jhu/county/010...,https://coronavirus.jhu.edu/static/media/dashb...,1597,35,422,551,841,1305,1515,74,30,5,2,24881,6788,433,34382,0,0,384184,4994,0,1997125,4634,22,2,25782,11898,12199,63,85,1,86,344,1106,12216,12266,72,96,1,778,353,24676,4517,3092,3675,1814,25782,1106,9864,2820,22.0,42.0,3.0,2.0,11.0,3.0,22.0,30.0,45.0,11.0,8.0,2.0,6.0,7.0,88.420883,56.711409,88.571429,3.258643e+09,307285.154510,"POLYGON ((-85.25609 32.13767, -85.25569 32.135..."
3,4,Bibb,Alabama,AL,01,01007,2.469136,8678.57,214.285714,21.8,4.0,92.3478,1.299898,01/07/2021 11:30:49,Govt Ordered Community Quarantine,4/3/2020 10:40:51 PM,AL Governor issued a Shelter In Place order.,https://bao.arcgis.com/covid-19/jhu/county/010...,https://coronavirus.jhu.edu/static/media/dashb...,1944,48,427,488,624,842,1280,35,25,4,1,22400,4400,344,46064,0,0,384184,4994,0,1997125,3661,21,2,22527,16801,4974,8,37,0,0,160,547,17268,5018,8,37,0,9,187,21980,3742,3005,3075,1539,22527,547,9044,2122,21.0,38.0,3.0,19.0,9.0,20.0,17.0,25.0,30.0,16.0,7.0,14.0,14.0,28.0,93.750000,19.047619,NaN,2.310715e+09,227886.963840,"POLYGON ((-87.02685 33.24646, -87.02572 33.209..."
4,5,Blount,Alabama,AL,01,01009,1.367905,8468.19,115.836791,13.2,3.5,101.0645,1.299898,01/07/2021 11:30:49,Govt Ordered Community Quarantine,4/3/2020 10:40:51 PM,AL Governor issued a Shelter In Place order.,https://bao.arcgis.com/covid-19/jhu/county/010...,https://coronavirus.jhu.edu/static/media/dashb...,4898,67,866,1459,1776,2802,3330,25,25,6,2,57840,7527,878,50412,0,0,384184,4994,

- FatalityRa = County Fatality Rate
- Confirmedb = County Confirmed divided by County Population * 100,000
- DeathsbyPo = County Deaths divided by County Population * 100,000
- Unemployme = Unemployment Rate
- EM_type, EM_date, EM_notes - could be turned into a column about Emergency Declaration
- Confirmed = County Level Confirmed Cases
- Deaths = County Level Deaths
- Beds_Licen = Number of Licensed Beds 
- Beds_Staff = Number of Staffed Beds 
- Beds_ICU = Number of ICU Beds 
- Ventilator = Average Ventilation Used Per Hospital
- POP_ESTIMA = Total Population
- POVALL_201 = Poverty Rate
- Median_Hou - Median Household Income
- Recovered = Number of Recovered Cases
- Active = Number of Active Cases
- AgedPop = TOtal Population Aged 65 Plus
- NewCases = New cases since yesterday
- NewDeaths = Newdeaths since yesterday
- Wh_Alone, Bk_Alone, AI_Alone, As_Alone, NH_Alone, SO_Alone, Two_More, Not_Hisp = Population numbers of each group
- Age_? = Population numbers of age groups
- NewCasebyP = I think this is new case by percentage
- ICU_Occ, Inpat_Occ = Percent Occupied of ICUs and Inpatient Beds
- Shape_Area, Shape_Length = for mapping



In [76]:
# remove all the rows with Unassigned and Out of [State] designations for county name, as well as Puerto Rico
data2 = data2[~data2.Countyname.str.contains("Out of")]
data2 = data2[~data2.Countyname.str.contains("Unassigned")]
data2 = data2[~data2.ST_Name.str.contains("Puerto Rico")]
data2.drop(data2.tail(7).index, inplace=True)
# remove Day_1 to Day_14 columns, unclear what the days represent
data2.drop(columns=['Day_1', 'Day_2', 'Day_3', 'Day_4', 'Day_5', 'Day_6', 'Day_7', 'Day_8', 'Day_9', 'Day_10', 'Day_11', 'Day_12', 'Day_13', 'Day_14'], inplace=True)
# State level data columns were dropped, as well as any ID and repeated population data
data2.drop(columns=['OBJECTID', 'ST_ID', 'PCTPOVALL_', 'Med_HH_Inc', 'State_Fata', 'DateChecke', 'url', 'Thumbnail', 'State_Conf', 'State_Deat', 'State_Reco', 'State_Test', 'TotalPop', 'NonHispWhP', 'BlackPop', 'AmIndop', 'PacIslPop', 'OtherPop', 'TwoMorPop', 'HispPop', 'NonHisp', 'AsianPop'], inplace=True)
data2.drop(columns=['Wh_Alone', 'Bk_Alone', 'AI_Alone', 'As_Alone', 'NH_Alone', 'SO_Alone', 'Two_More', 'Not_Hisp', 'Age_Over75'], inplace=True)
# Kept Deaths per 
data2.drop(columns=['Confirmed', 'Deaths'], inplace=True)
# remove all age category data, as there is not statistical difference between the various age groups above 65
data2.drop(columns=['Age_85', 'Age_80_84', 'Age_75_79', 'Age_70_74', 'Age_65_69', 'Agetotal', 'Age_Less15', 'Age_15_24', 'Age_25_34', 'Age_35_64', 'Age_65_74'], inplace=True)
# Columns are mostly empty values and difficult to categorize
data2.drop(columns=['Recovered', 'Active', 'EM_notes'], inplace=True)

In [77]:
data2.head()

,Countyname,ST_Name,ST_Abbr,FIPS,FatalityRa,Confirmedb,DeathsbyPo,Unemployme,EM_type,EM_date,Beds_Licen,Beds_Staff,Beds_ICU,Ventilator,POP_ESTIMA,POVALL_201,Unemployed,Median_Hou,AgedPop,NewCases,NewDeaths,NewCasebyP,Inpat_Occ,ICU_Occ,Shape__Area,Shape__Length,geometry
0,Autauga,Alabama,AL,01001,1.076426,8354.17,89.926440,3.6,Govt Ordered Community Quarantine,4/3/2020 10:40:51 PM,85,55,6,2,55601,7587,942,59338,8050,99,0,178.054352,96.320346,100.000000,2.209382e+09,246839.865479,"POLYGON ((-86.41312 32.70739, -86.41219 32.526..."
1,Baldwin,Alabama,AL,01003,1.166758,6722.26,78.432452,3.6,Govt Ordered Community Quarantine,4/3/2020 10:41:19 PM,386,362,51,8,218022,21069,3393,57588,40665,216,2,99.072571,74.029401,120.089286,5.770469e+09,728445.072448,"MULTIPOLYGON (((-87.78878 31.29877, -87.78849 ..."
2,Barbour,Alabama,AL,01005,2.191609,6418.55,140.669587,5.2,Govt Ordered Community Quarantine,4/3/2020 10:43:21 PM,74,30,5,2,24881,6788,433,34382,4634,22,2,88.420883,56.711409,88.571429,3.258643e+09,307285.154510,"POLYGON ((-85.25609 32.13767, -85.25569 32.135..."
3,Bibb,Alabama,AL,01007,2.469136,8678.57,214.285714,4.0,Govt Ordered Community Quarantine,4/3/2020 10:40:51 PM,35,25,4,1,22400,4400,344,46064,3661,21,2,93.750000,19.047619,NaN,2.310715e+09,227886.963840,"POLYGON ((-87.02685 33.24646, -87.02572 33.209..."
4,Blount,Alabama,AL,01009,1.367905,8468.19,115.836791,3.5,Govt Ordered Community Quarantine,4/3/2020 10:40:51 PM,25,25,6,2,57840,7527,878,50412,10233,49,4,84.716459,76.190476,92.857143,2.456058e+09,286306.840721,"POLYGON ((-86.44507 34.24954, -86.40902 34.205..."


In [79]:
data2.columns

Index(['Countyname', 'ST_Name', 'ST_Abbr', 'FIPS', 'FatalityRa', 'Confirmedb',
       'DeathsbyPo', 'Unemployme', 'EM_type', 'EM_date', 'Beds_Licen',
       'Beds_Staff', 'Beds_ICU', 'Ventilator', 'POP_ESTIMA', 'POVALL_201',
       'Unemployed', 'Median_Hou', 'AgedPop', 'NewCases', 'NewDeaths',
       'NewCasebyP', 'Inpat_Occ', 'ICU_Occ', 'Shape__Area', 'Shape__Length',
       'geometry'],
      dtype='object')

In [81]:
data2.head()

,Countyname,ST_Name,ST_Abbr,FIPS,FatalityRa,Confirmedb,DeathsbyPo,Unemployme,EM_type,EM_date,Beds_Licen,Beds_Staff,Beds_ICU,Ventilator,POP_ESTIMA,POVALL_201,Unemployed,Median_Hou,AgedPop,NewCases,NewDeaths,NewCasebyP,Inpat_Occ,ICU_Occ,Shape__Area,Shape__Length,geometry
0,Autauga,Alabama,AL,01001,1.076426,8354.17,89.926440,3.6,Govt Ordered Community Quarantine,4/3/2020 10:40:51 PM,85,55,6,2,55601,7587,942,59338,8050,99,0,178.054352,96.320346,100.000000,2.209382e+09,246839.865479,"POLYGON ((-86.41312 32.70739, -86.41219 32.526..."
1,Baldwin,Alabama,AL,01003,1.166758,6722.26,78.432452,3.6,Govt Ordered Community Quarantine,4/3/2020 10:41:19 PM,386,362,51,8,218022,21069,3393,57588,40665,216,2,99.072571,74.029401,120.089286,5.770469e+09,728445.072448,"MULTIPOLYGON (((-87.78878 31.29877, -87.78849 ..."
2,Barbour,Alabama,AL,01005,2.191609,6418.55,140.669587,5.2,Govt Ordered Community Quarantine,4/3/2020 10:43:21 PM,74,30,5,2,24881,6788,433,34382,4634,22,2,88.420883,56.711409,88.571429,3.258643e+09,307285.154510,"POLYGON ((-85.25609 32.13767, -85.25569 32.135..."
3,Bibb,Alabama,AL,01007,2.469136,8678.57,214.285714,4.0,Govt Ordered Community Quarantine,4/3/2020 10:40:51 PM,35,25,4,1,22400,4400,344,46064,3661,21,2,93.750000,19.047619,NaN,2.310715e+09,227886.963840,"POLYGON ((-87.02685 33.24646, -87.02572 33.209..."
4,Blount,Alabama,AL,01009,1.367905,8468.19,115.836791,3.5,Govt Ordered Community Quarantine,4/3/2020 10:40:51 PM,25,25,6,2,57840,7527,878,50412,10233,49,4,84.716459,76.190476,92.857143,2.456058e+09,286306.840721,"POLYGON ((-86.44507 34.24954, -86.40902 34.205..."


In [83]:
data2.describe()

,FatalityRa,Confirmedb,DeathsbyPo,Unemployme,Beds_Licen,Beds_Staff,Beds_ICU,Ventilator,POP_ESTIMA,POVALL_201,Unemployed,Median_Hou,AgedPop,NewCases,NewDeaths,NewCasebyP,Inpat_Occ,ICU_Occ,Shape__Area,Shape__Length
count,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3142.000000,3.142000e+03,3.142000e+03,3142.000000,3142.000000,3.142000e+03,3142.000000,3142.000000,3142.000000,2423.000000,1552.000000,3.142000e+03,3.142000e+03
mean,1.750076,6948.591687,122.715199,4.132018,298.811903,254.153405,29.178867,7.380013,1.041271e+05,1.332028e+04,2003.960853,52777.610439,1.509336e+04,80.888288,1.239338,81.933501,53.976173,65.560210,6.808623e+09,3.714886e+05
std,1.112969,2837.637191,89.957044,1.503802,985.479124,862.311711,102.185808,23.534068,3.334863e+05,4.559631e+04,6969.042922,13909.834889,4.499390e+04,316.806914,5.901344,82.248343,34.142614,29.709178,5.556968e+10,7.397408e+05
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,-41.000000,-5.000000,-1416.234888,0.000000,0.000000,8.763938e+06,1.563678e+04
25%,1.010014,5149.432500,60.155273,3.100000,16.000000,15.000000,2.000000,0.000000,1.092650e+04,1.594000e+03,200.250000,43679.500000,8.807500e+02,6.000000,0.000000,38.213398,34.330776,50.339860,1.860524e+09,2.058000e+05
50%,1.537125,6825.790000,105.698029,3.900000,42.000000,32.000000,5.000000,2.000000,2.575850e+04,3.766000e+03,482.500000,50566.500000,4.080000e+03,19.000000,0.000000,68.450955,54.614412,72.340426,2.680092e+09,2.493543e+05
75%,2.288272,8529.000000,161.625494,4.800000,171.000000,134.000000,14.000000,4.000000,6.782050e+04,9.391250e+03,1275.750000,58845.500000,1.161875e+04,55.000000,1.000000,107.267763,72.141680,88.000071,3.944540e+09,3.367670e+05
max,13.043478,28439.740000,842.266462,19.900000,23579.000000,22419.000000,2352.000000,523.000000,1.010552e+07,1.409155e+06,239829.000000,140382.000000,1.299277e+06,11554.000000,257.000000,1564.455569,1218.285714,200.000000,2.228678e+12,1.796423e+07


<img src='images/county_conf_by_pop.png'>

<img src='images/county_confirmed.png'>

## 2020 Election Data

In [71]:
# 
df2020 = pd.read_csv('data/2020_Results.csv')
df2020.county_fips = df2020.county_fips.astype(str)
df2020.county_fips = df2020.county_fips.apply(lambda x: x.zfill(5))
# upload shapefile for USA counties
shapefile = 'data/USA_Counties.shp'
gdf = gpd.read_file(shapefile)
geo = gdf[~gdf.STATE_NAME.isin(['Puerto Rico', 'Alaska', 'Hawaii'])]
geo.FIPS = geo.FIPS.astype(str)
merged = geo.merge(df2020, left_on='FIPS', right_on='county_fips', how='left')

In [73]:
# fig, ax = plt.subplots(1, figsize=(20,20))
# merged.plot(column='per_gop', cmap='RdBu_r', linewidth=0.8, ax=ax, edgecolor='black')
# ax.axis('off')
# ax.set_title('Percentage By County for 2020 Election', fontsize=18)
# plt.savefig('images/election2020.png')

<img src='images/election2020.png'>

In [ ]:
data2['Staf_Bed_Perc'] = data2.Beds_Staff / data2.Beds_Licen
data2['Aged_Perc'] = data2.AgedPop / data2.POP_ESTIMA
data2['Pov_Perc'] = data2.POVALL_201 / data2.POP_ESTIMA
data2['Aged_Per_ICU'] = data2.AgedPop / data2.Beds_ICU
data2.drop(columns=['Unemployed', 'Beds_Licen', 'Beds_Staff', 'Ventilator'], inplace=True)

data2.EM_date = data2.EM_date.str.extract(r'((\d+)\/(\d{2})\/(\d{4}))')
data2.EM_type = data2.EM_type.replace({'Govt Ordered Community Quarantine': 'CQ', 'Govt Directed Social Distancing': 'SD'})